In [ ]:
# ==========================================================================================
# BAGIAN 1: SETUP UTAMA
# Mengimpor library penting: Flask (Web), YOLO (AI), dan OpenCV (Kamera).
# ==========================================================================================
from flask import Flask, render_template, Response, request, redirect, url_for, session, jsonify
from ultralytics import YOLO
import cv2
import os
import time
from datetime import datetime

app = Flask(__name__)
app.secret_key = 'kunci_rahasia_ecoearth_2025' 

# --- Konfigurasi Sumber Kamera ---
# Daftar kamera yang bisa digunakan (0=Default, URL=IP Camera HP)
IP_CAMERA_URL = 'http://192.168.1.8:8080/video' 
CAMERA_SOURCES = [1, 2, IP_CAMERA_URL]          
current_camera_index = 0                        

# --- Memuat Model AI ---
# Kita mencoba meload model 'best.pt' hasil training sendiri. Jika gagal, pakai model standar.
latest_result = {'kategori': 'Waiting...', 'nama_id': '...', 'nama_en': '...', 'akurasi': '0%'}

print(" Sedang memuat model AI...")
try:
    model = YOLO('best.pt') 
    print(" BERHASIL: Menggunakan model custom 'best.pt'")
except:
    print(" Gagal load best.pt, menggunakan yolov8n.pt")
    model = YOLO('yolov8n.pt')

# ==========================================================================================
# BAGIAN 2: LOGIKA LOGIKA PENTING (WARNA & KATEGORI)
# Menentukan warna kotak dan jenis sampah (Organik/Anorganik) berdasarkan nama benda.
# ==========================================================================================

# Warna BGR untuk kotak deteksi (Hijau, Kuning, Biru)
COLOR_GREEN = (94, 197, 34)   
COLOR_YELLOW = (8, 179, 234)  
COLOR_BLUE = (246, 130, 59)   

def get_color_by_category(category):
    if category == 'Organic': return COLOR_GREEN
    if category == 'Anorganic': return COLOR_YELLOW
    if category == 'Non-Trash': return COLOR_BLUE
    return (0, 255, 0) 

def tentukan_kategori(nama_kelas):
    # Fungsi ini mengecek apakah nama benda mengandung kata 'organic', 'anorganic', atau 'not trash'
    nama_lower = nama_kelas.lower()
    
    if 'organic' in nama_lower or 'organik' in nama_lower:
        if 'anorganik' not in nama_lower and 'anorganic' not in nama_lower:
            return 'Organic'

    if 'anorganik' in nama_lower or 'anorganic' in nama_lower:
        return 'Anorganic'

    if 'not trash' in nama_lower or 'manusia' in nama_lower:
        return 'Non-Trash'

    return 'Anorganic' # Default

# ==========================================================================================
# BAGIAN 3: PEMROSESAN VIDEO (CORE AI LOGIC)
# Fungsi ini menerima gambar dari kamera, mendeteksi objek, dan menggambarnya.
# ==========================================================================================

def process_frame(frame):
    global latest_result
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = model(frame_rgb, verbose=False, conf=0.10) 
    
    temp_result = None
    h, w, _ = frame.shape
    font_scale = 1.0 if w > 800 else 0.6
    thickness = 2

    for r in results:
        for box in r.boxes:
            cls_id = int(box.cls[0])
            raw_label = model.names[cls_id]
            conf = int(float(box.conf[0]) * 100)
            
            if raw_label == '0': continue # Abaikan jika deteksi kosong/salah
            
            # 1. Tentukan kategori dan warna kotak
            kategori = tentukan_kategori(raw_label)
            color = get_color_by_category(kategori)
            
            # 2. Ambil nama benda (ID dan EN) dari label dataset (misal: Kaca_Glass_Anorganik)
            parts = raw_label.split('_')
            nama_id = parts[0]
            nama_en = parts[1] if len(parts) > 1 else parts[0]
            
            # 3. Gambar kotak (Rectangle) dan Teks di layar
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness)
            
            display_text = f"{nama_id} {conf}%"
            (tw, th), _ = cv2.getTextSize(display_text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, thickness)
            
            # Background teks agar tulisan terbaca jelas
            if y1 - th - 10 < 0:
                cv2.rectangle(frame, (x1, y1), (x1 + tw + 10, y1 + th + 15), color, -1)
                text_pos = (x1 + 5, y1 + th + 10)
            else:
                cv2.rectangle(frame, (x1, y1 - th - 15), (x1 + tw + 10, y1), color, -1)
                text_pos = (x1 + 5, y1 - 5)
                
            cv2.putText(frame, display_text, text_pos, cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), thickness)
            
            # 4. Simpan hasil ke variabel global untuk dikirim ke Dashboard
            temp_result = {
                "nama_id": nama_id, 
                "nama_en": nama_en, 
                "kategori": kategori, 
                "akurasi": f"{conf}%"
            }
            
    if temp_result: latest_result = temp_result
    return frame

# ==========================================================================================
# BAGIAN 4: STREAMING & RUTE WEBSITE
# Mengatur pengiriman gambar ke browser dan halaman-halaman website (Home, Login, Dashboard)
# ==========================================================================================

def generate_frames():
    # Fungsi ini terus menerus mengambil gambar dari kamera dan mengirimnya ke browser
    global current_camera_index
    source = CAMERA_SOURCES[current_camera_index % len(CAMERA_SOURCES)]
    
    if isinstance(source, int): cap = cv2.VideoCapture(source, cv2.CAP_DSHOW)
    else: cap = cv2.VideoCapture(source)

    frame_counter = 0
    while True:
        success, frame = cap.read()
        if not success: break
        
        frame_counter += 1
        if frame_counter % 3 != 0: continue # Skip frame biar ringan

        try: frame_resized = cv2.resize(frame, (640, 480))
        except: continue
        
        processed_frame = process_frame(frame_resized)
        ret, buffer = cv2.imencode('.jpg', processed_frame)
        yield (b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + buffer.tobytes() + b'\r\n')
    
    cap.release()

# --- Daftar Halaman (Routes) ---

@app.route('/')
def landing(): return render_template('index.html')

@app.route('/login', methods=['GET', 'POST'])
def login_page():
    if request.method == 'POST':
        session['logged_in'] = True
        return redirect(url_for('dashboard'))
    return render_template('login.html')

@app.route('/video_feed')
def video_feed(): return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/status_feed')
def status_feed(): return jsonify(latest_result) # API untuk data teks live

@app.route('/switch_camera')
def switch_camera():
    global current_camera_index
    current_camera_index = (current_camera_index + 1) % len(CAMERA_SOURCES)
    return redirect(url_for('dashboard', tab='live'))

@app.route('/dashboard', methods=['GET', 'POST'])
def dashboard():
    if request.method == 'POST':
        if 'back_to_menu' in request.form: return redirect(url_for('dashboard', tab='menu'))
    tab = request.args.get('tab', 'menu')
    return render_template('dashboard.html', active_tab=tab, cam_index=current_camera_index, history=session.get('history', []))

@app.route('/logout')
def logout():
    session.pop('logged_in', None)
    return redirect(url_for('landing'))

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5005, debug=False)

 Sedang memuat model AI...
 BERHASIL: Menggunakan model custom 'best.pt'
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5005
 * Running on http://10.10.11.200:5005
Press CTRL+C to quit
127.0.0.1 - - [15/Jan/2026 08:30:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2026 08:30:38] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [15/Jan/2026 08:30:40] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2026 08:30:49] "POST /login HTTP/1.1" 302 -
127.0.0.1 - - [15/Jan/2026 08:30:49] "GET /dashboard HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2026 08:31:34] "GET /dashboard?tab=live HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2026 08:31:34] "GET /status_feed HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2026 08:31:35] "GET /status_feed HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2026 08:31:35] "GET /video_feed HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2026 08:31:35] "GET /status_feed HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2026 08:31:36] "GET /status_feed HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2026 08:31:36] "GET /status_feed HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2026 08:31